In [1]:
import logging
import os
from datetime import datetime as dt
from pathlib import Path
from typing import Dict

import numpy as np
import ray
import torch
from ray import train, tune
from torchvision import transforms

from engine import (load_enet_best, get_loss_optimizer, create_writer, pretty_json)
from engine import train as train_enet
from load_data_functions import create_cv_datasets, create_dataloaders

In [2]:
project_path = os.getcwd()

def tune_cv_enet(config: Dict[str, any], start_time_tuning: str):
    """
    Tune the pretrained EfficientNet model based on the given hyperparameter configuration using cross validation and
    log the results using ray train.report and Tensorboard.

    :param config:  Dictionary containing hyperparameters for tuning.
        - "bs" (int): Batch size.
        - "do" (float): Dropout probability for the dropout layer.
        - "loss_fn" (str): Name of the loss function. 
        - "lr" (float): Learning rate.
        - "aug" (bool): Whether to apply data augmentation to increase the training set size.
    :param start_time_tuning: Start time of the tuning run for logging purposes.
    """
    # Extract hyperparameters from the config
    batch_size = config["bs"]
    dropout_p = config["do"]
    loss_fn_str = config["loss_fn"]
    learning_rate = config["lr"]
    augmentation = config["aug"]

    # fixed parameters and other variables
    split_path = Path(project_path) / "data/train_test_split"
    fold_list = [split_path / f"fold_{i}" for i in range(1, 6)]
    used_fold = "all"
    img_size = 224
    model_name = "pretrained_enet"
    optimizer_str = "Adam"
    num_epochs = 20
    train_percentage = 1.0

    # setup device agnostic code
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    # create a writer to track training results
    writer = create_writer(f"hp_tuning_cv", model_name, f"start_{start_time_tuning}", project_path)

    # track hyperparameters and other relevant information
    info = {
        "model_name": model_name,
        "batch_size": batch_size,
        "dropout_p": dropout_p,
        "loss_fn_str": loss_fn_str,
        "learning_rate": learning_rate,
        "augmentation": augmentation,
        "split_path": str(split_path),
        "fold_list": [str(fold) for fold in fold_list],
        "validation_fold": used_fold,
        "fold_num": len(fold_list),
        "img_size": img_size,
        "num_epochs": num_epochs,
        "optimizer_str": optimizer_str,
        "percentage": train_percentage,
        "cross_validation": True
    }

    writer.add_text("Miscellaneous", pretty_json(info))

    # define data transforms
    data_transforms = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    augment_transform = transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),  # resize the images
        # randomly add Gaussian blur
        transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.7),
        # randomly perform a horizontal flip
        transforms.RandomHorizontalFlip(p=0.7),
        # turn the images into a torch.Tensor
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    # perform cross validation
    datasets = create_cv_datasets(fold_list, data_transforms, select_one=False,
                                  train_percentage=train_percentage,
                                  augmentation=augmentation, augment_transform=augment_transform)
    cv_results = {}
    for fold in datasets.keys():
        train_data = datasets[fold]["train"]
        validation_data = datasets[fold]["validation"]
        print("\n-------------------------------------------------------------------------")
        print(f"Size of the training set: {len(train_data)}")
        print(f"Size of the validation set: {len(validation_data)}")
        train_loader, val_loader = create_dataloaders(train_data, validation_data, device, batch_size)

        # initialize model and optimizer
        model = load_enet_best(device, dropout=dropout_p, project_path=project_path)
        model.to(device)
        loss_fn, optimizer = get_loss_optimizer(loss_fn_str, optimizer_str, model, learning_rate)

        # train the model
        print(f"Training results for {fold}:")
        cv_results[fold] = train_enet(num_epochs=num_epochs,
                                      model=model,
                                      train_loader=train_loader,
                                      val_loader=val_loader,
                                      loss_fn=loss_fn,
                                      optimizer=optimizer,
                                      device=device,
                                      writer=writer,
                                      fold=fold)

    # initialize lists to store epoch-wise averages
    epoch_avg_ccc = []
    epoch_avg_pcc = []
    epoch_avg_loss = []

    # calculate cross validation performance
    for epoch in range(num_epochs):
        # extract validation and training metrics for all folds
        fold_val_cccs = [perf["val_ccc"][epoch] for perf in cv_results.values()]
        fold_val_pccs = [perf["val_pcc"][epoch] for perf in cv_results.values()]
        fold_val_loss = [perf["val_loss"][epoch] for perf in cv_results.values()]
        fold_train_cccs = [perf["train_ccc"][epoch] for perf in cv_results.values()]
        fold_train_pccs = [perf["train_pcc"][epoch] for perf in cv_results.values()]
        fold_train_loss = [perf["train_loss"][epoch] for perf in cv_results.values()]

        # calculate mean metrics for validation and training
        mean_val_ccc = np.mean(fold_val_cccs)
        mean_val_pcc = np.mean(fold_val_pccs)
        mean_val_loss = np.mean(fold_val_loss)
        mean_train_ccc = np.mean(fold_train_cccs)
        mean_train_pcc = np.mean(fold_train_pccs)
        mean_train_loss = np.mean(fold_train_loss)

        # append to epoch-wise lists
        epoch_avg_ccc.append(mean_val_ccc)
        epoch_avg_pcc.append(mean_val_pcc)
        epoch_avg_loss.append(mean_val_loss)

        writer.add_scalars(main_tag="avg_ccc",
                           tag_scalar_dict={"mean_val_ccc": mean_val_ccc,
                                            "mean_train_ccc": mean_train_ccc},
                           global_step=epoch)
        writer.add_scalars(main_tag="avg_pcc",
                           tag_scalar_dict={"mean_val_pcc": mean_val_pcc,
                                            "mean_train_pcc": mean_train_pcc},
                           global_step=epoch)
        writer.add_scalars(main_tag="avg_loss",
                           tag_scalar_dict={"mean_val_loss": mean_val_loss,
                                            "mean_train_loss": mean_train_loss})

    # evaluate and report results with ray
    best_ccc = max(epoch_avg_ccc)
    best_epoch = epoch_avg_ccc.index(best_ccc)
    best_epoch_pcc = epoch_avg_pcc[best_epoch]
    best_epoch_loss = epoch_avg_loss[best_epoch]
    train.report({"best_epoch": best_epoch, "best_avg_ccc": best_ccc,
                  "val_avg_pcc": best_epoch_pcc, "val_avg_loss": best_epoch_loss})

    # log hyperparameters and metrics for the best epoch (in terms of ccc)
    hparam_dict = {f"{key}": value for key, value in config.items()}
    hparam_dict["best_epoch"] = best_epoch
    metric_dict = {"hparam/best_val_ccc": best_ccc,
                   "hparam/val_pcc": best_epoch_pcc,
                   "hparam/val_loss": best_epoch_loss}
    writer.add_hparams(hparam_dict, metric_dict)

In [3]:
search_space = {
    "bs": 256,
    "do": tune.uniform(0.0, 0.5),
    "loss_fn": "MSE",
    "lr": tune.quniform(1e-4, 1e-3, 5e-5),
    "aug": tune.choice([True, False])
}

os.environ["TUNE_DISABLE_AUTO_CALLBACK_LOGGERS"] = "1"

ray.shutdown()
ray.init(num_gpus=1, logging_level=logging.ERROR)
resources = {"gpu": 1}
trainable_with_resources = tune.with_resources(tune_cv_enet, resources)

start_tune = dt.now().strftime("%H-%M-%S")

tuner = tune.Tuner(
    tune.with_parameters(trainable_with_resources, start_time_tuning=start_tune),
    tune_config=tune.TuneConfig(num_samples=7),
    param_space=search_space
)

results = tuner.fit()

(tune_cv_enet pid=24456) cuda
(tune_cv_enet pid=24456) [INFO] Created SummaryWriter, saving to: C:\local\AIProject\runs\hp_tuning_cv\pretrained_enet\start_07-11-56\2024-05-30_07-12-01...
(tune_cv_enet pid=24456) 
(tune_cv_enet pid=24456) -------------------------------------------------------------------------
(tune_cv_enet pid=24456) Size of the training set: 39606
(tune_cv_enet pid=24456) Size of the validation set: 4915


  0%|          | 0/20 [00:00<?, ?it/s]


(tune_cv_enet pid=24456) [INFO] Created a new pretrained EfficientNet-B0 model.
(tune_cv_enet pid=24456) Training results for fold_1:


  5%|▌         | 1/20 [02:36<49:30, 156.32s/it]


(tune_cv_enet pid=24456) Epoch: 1 | train_loss: 0.0507 | train_ccc: 0.6089 | train_pcc: 0.6292 | val_loss: 0.0550 | val_ccc: 0.4430 | val_pcc: 0.5079


...

 90%|█████████ | 18/20 [44:34<04:57, 148.54s/it]


(tune_cv_enet pid=12720) Epoch: 18 | train_loss: 0.0252 | train_ccc: 0.8110 | train_pcc: 0.8266 | val_loss: 0.0473 | val_ccc: 0.5024 | val_pcc: 0.6224


 95%|█████████▌| 19/20 [47:04<02:28, 148.77s/it]


(tune_cv_enet pid=12720) Epoch: 19 | train_loss: 0.0246 | train_ccc: 0.8163 | train_pcc: 0.8305 | val_loss: 0.0477 | val_ccc: 0.5258 | val_pcc: 0.6246
(tune_cv_enet pid=12720) Epoch: 20 | train_loss: 0.0244 | train_ccc: 0.8166 | train_pcc: 0.8312 | val_loss: 0.0474 | val_ccc: 0.5248 | val_pcc: 0.6303


100%|██████████| 20/20 [49:32<00:00, 148.65s/it]
